In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Öncelikle ihityacımız olan kütüphaneleri ekliyoruz.

In [ ]:
!pip install chart_studio
import chart_studio.plotly as py # visualization library


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns # visualization library
import matplotlib.pyplot as plt # visualization library
from plotly.offline import init_notebook_mode, iplot # plotly offline mode
init_notebook_mode(connected=True) 
import plotly.graph_objs as go # plotly graphical object

from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
import operator

Datayı yükleyelim

In [ ]:
weather = pd.read_csv("../input/weatherww2/Summary of Weather.csv")
weather_station_location = pd.read_csv("../input/weatherww2/Weather Station Locations.csv")


Şimdi datamızı biraz tanımamız gerekiyor.


In [ ]:
weather.columns

In [ ]:
weather_station_location.columns

Weather:
- STA: weather station number (WBAN)(Weather Bureau, Air Force, and Navy)
- Date: Date of temperature measurement (Sıcaklık ölçüm tarihi)
- MeanTemp: Mean temperature (Ortalama Sıcaklık)

Weather station location:
- WBAN: Weather station number 
- NAME: weather station name
- STATE/COUNTRY ID: acronym of countries
- Latitude: Latitude of weather station (Meteoroloji istasyonunun enlemi)
- Longitude: Longitude of weather station (Meteoroloji istasyonunun boylamı)

Görüldüğü üzre datamızda birçok sütun var. Biz bunlardan sadece bir kaç tanesini kullanacağız. Bu yüzden sadece o kolonlardan oluşan yeni data frameler oluşturalım

In [ ]:
index0=["WBAN","NAME","STATE/COUNTRY ID","Latitude","Longitude"] 


sta_loc_df = weather_station_location.reindex(columns=index0 ,fill_value=0)
sta_loc_df.info()

In [ ]:
index1=["MaxTemp","MinTemp", "Date"]

wea_df = weather.reindex(columns=index1, fill_value=0)
wea_df.info()

In [ ]:
wea_df = wea_df[:][:500]    
wea_df.head(4)

In [ ]:
sta_loc_df.tail(4)

In [ ]:
wea_df.describe().T

In [ ]:
pd.to_datetime(wea_df['Date'])

In [ ]:
sns.pairplot(wea_df, kind="reg")

In [ ]:
# See picture with scatter or plot method



plt.figure(figsize=(22,10))
plt.plot(wea_df.Date, wea_df.MaxTemp, wea_df.MinTemp,)
plt.title("Max and Min Temperature of Dates")
plt.xlabel("Date")
plt.ylabel("Max and Min Temperature")
plt.legend()
plt.show()

In [ ]:

y = np.array(wea_df['MaxTemp']).reshape(-1, 1)
X = np.array(wea_df['MinTemp']).reshape(-1, 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 13)

#### Korelasyon Matrisi
Korelasyon bağımlı ve bağımsız değişken arasındaki ilişkinin gücünü açıklar.

In [ ]:
corr=wea_df.corr()
sns.heatmap(corr, cmap = 'Wistia', annot= True);

In [ ]:
corr1=sta_loc_df.corr()
sns.heatmap(corr1, cmap = 'Wistia', annot= True);

# R-Squared 
Konumuza başlamadan önce algoritmalarımızın hangisinin daha doğru sonuçlar vereceğini tespit etmemize yarayan R2 score'dan bahsetmek istiyorum.

**R² ,regresyon ile modelin ne kadar uyumlu olduğunu gösterir.**


\begin{aligned} &\text{R}^2 = 1 - \frac{ \text{Unexplained Variation} }{ \text{Total Variation} } \\ \end{aligned} 
​	  




   **Hata Kareleri Toplamı**(Verinin Varyasyonu)
   
    ∑(gercek deger-tahmin)²
    ∑( yi - y'i)²

   **Ortalama Farkların Toplamı**
   
   Burada OFT olarak bulduğumuz değer en kötü çıkabilecek ihtimaldir. Örneğin bir kilo tahmini yapılsa sınıf içerisinde, en kolay yol tüm sınıfın ortalama kilosunu bulup herkese o tahmini yapmaktır. 
   
    ∑(yi - y ort)² 

   **R²**
   
    R² =  1- ( HKT / OFT ) 

>R² değerleri 0 ile 1 arasındadır. 
En aptal olarak tanımlayabileceğimiz algoritmanın R-kare si 0 s-çıkacaktır.
%100 lük bir sonuç yani R²'nin 1 çıkması demek sonuçları tam olarak tahmin edebiliyoruz demektir. Fakat overfitting dediğimiz ezberleme durumunun gerçekleşme ihtimalinden ötürü böyle bir sonuç da istenmez. Bir modelin R² değeri 0,50 ise, gözlemlenen varyasyonun yaklaşık yarısı modelin girdileri ile açıklanabilir.

# 1. Linear Regression
II.Dünya Savaşında, günlük min ve max sıcaklık arasında nasılbir ilişki var?

In [ ]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()  
lin_reg.fit(X_train, y_train)

In [ ]:
y_pred = lin_reg.predict(X_test)                                     # Predict Linear Model
accuracy_score = lin_reg.score(X_test, y_test)                       # Accuracy score
print("Linear Regression Model Accuracy Score: " + "{:.1%}".format(accuracy_score))

In [ ]:
from sklearn.metrics import mean_squared_error,r2_score

lin_reg_r2=r2_score(y_test, y_pred)

print("R2 Score: " +"{:.3}".format(lin_reg_r2))




In [ ]:
import statsmodels.api as sm
model=sm.OLS(lin_reg.predict(X_test),X_test)
print(model.fit().summary())

In [ ]:
# Finally draw figure of Linear Regression Model

plt.scatter(X_test, y_test, color='r')
plt.plot(X_test, y_pred, color='g')
plt.show()

In [ ]:
from sklearn.linear_model import LinearRegression

X_train_val=X_train
y_train_val=y_train

lin_reg0 = LinearRegression()  
lin_reg0.fit(X_train_val, y_train_val)

plt.scatter(X_train_val ,y_train_val ,color="red")
plt.plot(X_train,lin_reg0.predict(X_train_val),color="blue")


# 2. Polynomial Regression

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

poly_reg=PolynomialFeatures(degree= 5)
x_poly=poly_reg.fit_transform(X_train)
print(x_poly)

lin_reg2=LinearRegression()
lin_reg2.fit(x_poly,y_train)

poly_pred=lin_reg2.predict(x_poly)

In [ ]:
rmse = np.sqrt(mean_squared_error(y_train,poly_pred))
poly_reg_r2 = r2_score(y_train,poly_pred)
print("RMSE Score for Test set: " +"{:.2}".format(rmse))
print("R2 Score for Test set: " +"{:.2}".format(poly_reg_r2))

In [ ]:
plt.scatter(X_train, y_train, s=50,color="red")
# sort the values of x before line plot
sort_axis = operator.itemgetter(0)
sorted_zip = sorted(zip(X_train,poly_pred), key=sort_axis)
X_train, poly_pred = zip(*sorted_zip)
plt.plot(X_train, poly_pred, color='blue')
plt.show()

# 3.Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeRegressor

dt_reg = DecisionTreeRegressor()          
dt_reg.fit(X_train,y_train)

In [ ]:
dt_predict = dt_reg.predict(X_train)

In [ ]:
plt.scatter(X_train,y_train, color="red")                           
X_grid = np.arange(min(np.array(X_train)),max(np.array(X_train)), 0.01)  
X_grid = X_grid.reshape((len(X_grid), 1))
plt.plot(X_grid,dt_reg.predict(X_grid),color="blue")
plt.xlabel("Temperature")
plt.ylabel("Salinity")
plt.title("Decision Tree Model")
plt.show()

In [ ]:
rmse = np.sqrt(mean_squared_error(y_train,dt_predict))
dt_reg_r2 = r2_score(y_train,dt_predict)
print("RMSE Score for Test set: " +"{:.2}".format(rmse))
print("R2 Score for Test set: " +"{:.2}".format(dt_reg_r2))

# 4. Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf_reg = RandomForestRegressor(n_estimators=6, random_state=0)
rf_reg.fit(X_train,y_train)
rf_predict = rf_reg.predict(X_train)


In [ ]:
plt.scatter(X_train,y_train, color="red")                          
X_grid = np.arange(min(np.array(X_train)),max(np.array(X_train)), 0.01)  
X_grid = X_grid.reshape((len(X_grid), 1))
plt.plot(X_grid,rf_reg.predict(X_grid),color="blue")             
plt.xlabel("Temperature")
plt.ylabel("Salinity")
plt.title("Random Forest Model")
plt.show()

In [ ]:
rmse = np.sqrt(mean_squared_error(y_train,rf_predict))
rf_reg_r2 = r2_score(y_train,rf_predict)
print("RMSE Score for Test set: " +"{:.2}".format(rmse))
print("R2 Score for Test set: " +"{:.2}".format(rf_reg_r2))

# 4. SVR
Doğru sonuç alabilmek için scale edilmesi gerekiyor değerlerin. 

In [ ]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
xTra_sc=sc.fit_transform(X_train)
yTra_sc=sc.fit_transform(y_train)
xTes_sc=sc.fit_transform(X_test)

In [ ]:
from sklearn.svm import SVR
svr_reg =SVR(kernel ="rbf") #diğer seçenekler: linear, poly
svr_reg.fit(xTra_sc,yTra_sc)

plt.scatter(xTra_sc,yTra_sc,color="red")
plt.plot(xTra_sc,svr_reg.predict(xTra_sc),color="blue")



In [ ]:
from sklearn.metrics import r2_score
# Haydi r2 değeri hesaplayalım.
print("SVR R2 değeri")
#olceki değerler scale'li olanlar alındı.

svr_reg_r2= r2_score(yTra_sc, svr_reg.predict(xTra_sc))
print(svr_reg_r2)


In [ ]:
print("Weather Prediction")
print("Linear: ")
print(lin_reg.predict([[10]]))
print("Polynomial: ")
print(lin_reg2.predict(poly_reg.fit_transform([[10]])))
print("SVR: rbf?")
print(svr_reg.predict([[10]]))
print("Decision Tree: ")
print(dt_reg.predict([[10]]))
print("Random Forest : ")
print(rf_reg.predict([[10]]))



In [ ]:

#%% R2 DEĞERLERİ KARŞILAŞTIRMA

print("···R2 DEĞERLERİ···")
print("Linear Regression     :     "+str(lin_reg_r2))
print("Polynomial Regression :     "+str(poly_reg_r2))
print("Decision Tree         :     "+str(dt_reg_r2))
print("Random Forest         :     "+str(rf_reg_r2))
print("SVR Regression        :     "+str(svr_reg_r2))
